# Lane Follower - CompRobo17

## Imports and Directories

In [1]:
#Create references to important directories we will use over and over
import os, sys
current_dir = os.getcwd()
SCRIPTS_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir+'/data'

In [2]:
#import modules
from utils import *
%matplotlib inline

Using TensorFlow backend.


In [3]:
%cd $DATA_HOME_DIR

#Set path to sample/ path if desired
path = DATA_HOME_DIR
train_path=path + '/train/binary/'
valid_path=path + '/valid/binary/'

/home/nathan/olin/spring2017/lane_follower/src/scripts/data


## Convnet

### Constants

In [4]:
img_rows, img_cols = 16, 32
in_shape = (img_rows, img_cols)
batch_size = 64
nb_classes = 3

### Batches

In [5]:
gen = image.ImageDataGenerator(rescale=1. / 255)

In [6]:
train_batches = get_batches(train_path, batch_size=batch_size, 
                            target_size=in_shape, color_mode="grayscale", 
                            gen=gen)
val_batches   = get_batches(valid_path, batch_size=batch_size, 
                            target_size=in_shape, color_mode="grayscale", 
                            gen=gen)

Found 938 images belonging to 3 classes.
Found 536 images belonging to 3 classes.


### Model

In [17]:
def get_model():
    model = Sequential([
        Convolution2D(32,3,3, border_mode='same', activation='relu', input_shape=(img_rows, img_cols, 1)),
        MaxPooling2D(),
        Convolution2D(64,3,3, border_mode='same', activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(1024, activation='relu'),
        Dropout(0.5),
        Dense(nb_classes, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [18]:
model = get_model()

In [19]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_6 (Convolution2D)  (None, 16, 32, 32)    320         convolution2d_input_4[0][0]      
____________________________________________________________________________________________________
maxpooling2d_6 (MaxPooling2D)    (None, 8, 16, 32)     0           convolution2d_6[0][0]            
____________________________________________________________________________________________________
convolution2d_7 (Convolution2D)  (None, 8, 16, 64)     18496       maxpooling2d_6[0][0]             
____________________________________________________________________________________________________
maxpooling2d_7 (MaxPooling2D)    (None, 4, 8, 64)      0           convolution2d_7[0][0]            
___________________________________________________________________________________________

### Train

In [20]:
model.fit_generator(train_batches, samples_per_epoch=train_batches.nb_sample,
                    nb_epoch=5,
                    validation_data=val_batches,
                    nb_val_samples=val_batches.nb_sample,
                    verbose=True)

Epoch 1/5
938/938 [==============================] - 1s - loss: 0.4915 - acc: 0.7932 - val_loss: 0.3242 - val_acc: 0.8806
Epoch 2/5
938/938 [==============================] - 0s - loss: 0.3581 - acc: 0.8699 - val_loss: 0.3405 - val_acc: 0.8470
Epoch 3/5
938/938 [==============================] - 0s - loss: 0.3198 - acc: 0.8817 - val_loss: 0.3212 - val_acc: 0.8489
Epoch 4/5
938/938 [==============================] - 0s - loss: 0.3045 - acc: 0.8838 - val_loss: 0.3184 - val_acc: 0.8937
Epoch 5/5
938/938 [==============================] - 0s - loss: 0.2915 - acc: 0.8817 - val_loss: 0.3518 - val_acc: 0.8526


In [21]:
model.save_weights('binary_86.h5')

In [23]:
train_batches.class_indices

{'forward': 0, 'left': 1, 'right': 2}